In [47]:
import numpy as np
import pandas as pd
from bigtree import Node,tree_to_dict
from random import randint


In [48]:
#Fonctin complémentaire
#Retirer le 1 ou deux du traitement pour éviter de créer un ordre de traitement
def retirerChiffre(string):
    k=list(string)
    i = 0
    while k[i].isdigit()==False:
        i+=1
    k.remove(k[i])
    u=''.join(k)
    return u


In [49]:
#nombre de traitement maximal à prendre en compte pour l'entrainement de l'IA : en commun avec le fichier de préparation des données
t_max = 2

Partie 1 : récupération et traitement des données d'entraînement

In [50]:
# Table de faux patients
#On récupère les données issues du fichier "preparation_donnee.ipynb" : notre dataframe d'entrainement
#Dans ce DataFrame, des conflits iatrogéniques que l'on va nommer "règles", sont noyés parmi de nombreux cas négatifs

df = pd.read_csv("patientFrame_N.csv")

colonne = []
for i in range(t_max):
    colonne.append("treatment"+str(i+1)) 

colonne.append('iatrogenie')

information_table = df[colonne]
#Affichage des données obtenues
information_table

,treatment1,treatment2,iatrogenie
0,acetazolamide=true,abiraterone=true,0
1,acide_acetylsaclicylique>=3g,abiraterone=true,0
2,acide_acetylsaclicylique=true,abiraterone=true,0
3,acide_acetylsaclicylique>=3g,metoprolol=true,0
4,acide_acetylsaclicylique=true,fluorouracil=true,0
...,...,...,...
495,acetazolamide=true,anti-inflammatoire_non_stéroïdiens,0
496,acide_folinique=true,fluorouracil=true,1
497,acide_acetylsaclicylique=true,anti-inflammatoire_non_stéroïdiens,3
498,acetazolamide=true,carbamazepine=true,1


Partie II : Implémentation de fonctions nécessaires à l'algorithme de construction de l'arbre granulaire

In [51]:
#Fonction de création des différents granules
#Granule =[Formule, Regroupement des données associées à la formule]
def getFormulasGranules(column_attr, info_table):
    
    # Granule de base correspondant à une formule F et les éléments du granule G
    basic_granules_table = pd.DataFrame(columns = ['Formula', 'Granule'])
    
    # Index i
    i = 0
    
    for attr in column_attr:
        group = info_table.groupby( by = attr ).groups
        keys = list(group.keys())
        values = list(group.values())

        for key in keys:
            basic_granules_table.loc[i, 'Formula'] = attr + "=" + key
            basic_granules_table.loc[i, 'Granule'] = list(group[key])
            i += 1
        
    return basic_granules_table

In [52]:
# Définition et ajout de la mesure "G : Generality" dans la table des granules
# G = nombres de granules / cardinal de l'univers
def getGenerality(basic_gr_table, info_table):
    for i in range(len(basic_gr_table)):
        # nombres d'éléments dans le granule au rang i
        obj_in_granule =  len(basic_gr_table.loc[i, 'Granule'])
        U = len(info_table)
        #on insère une colonne de "Generality" dans laquelle on renseigne la mesure au rang i
        basic_gr_table.loc[i, 'Generality'] = obj_in_granule / U

In [53]:
#Fonction pour ajouter la mesure de "Confidence"

#pour implémenter cette mesure, nécessaire de déf une fonction qui compte les
# éléments de chaque classe
#adapter en fonction du nombre de classe définies et possibles 

def countClasseGr(objects_Gr, info_table):
    class_0 = 0
    class_1 = 0
    class_2 = 0
    class_3 = 0
    
    for i in objects_Gr:
        if info_table.loc[i, 'iatrogenie'] == 0:
            class_0 += 1
        if info_table.loc[i, 'iatrogenie'] == 1:
            class_1 += 1
        elif info_table.loc[i, 'iatrogenie'] == 2:
            class_2 += 1
        elif info_table.loc[i, 'iatrogenie'] == 3:
            class_3 += 1

    
    return class_0, class_1,class_2,class_3

def getConfidence( basic_gr_table, info_table ):
    
    
    for i in range(len(basic_gr_table)):
        obj_Gr = basic_gr_table.loc[i, 'Granule']
    
        class_0, class_1,class_2, class_3  = countClasseGr(obj_Gr, info_table)
    
        basic_gr_table.loc[i, 'confidence_0']  = class_0 / len(obj_Gr)
        basic_gr_table.loc[i, 'confidence_1']  = class_1 / len(obj_Gr)
        basic_gr_table.loc[i, 'confidence_2']  = class_2 / len(obj_Gr)
        basic_gr_table.loc[i, 'confidence_3']  = class_3 / len(obj_Gr)

In [54]:
# Mesure du coverage
def getCoverage(basic_gr_table, info_table):
    if 0 in info_table[['iatrogenie']].values:
        class_0_count = len(info_table.groupby( by = 'iatrogenie').groups[0])
    else:
        class_0_count = 0

    if 1 in info_table[['iatrogenie']].values:
        class_1_count = len(info_table.groupby( by = 'iatrogenie').groups[1])
    else:
        class_1_count = 0

    if 2 in info_table[['iatrogenie']].values:
        class_2_count = len(info_table.groupby( by = 'iatrogenie').groups[2])
    else:
        class_2_count = 0

    if 3 in info_table[['iatrogenie']].values:
        class_3_count = len(info_table.groupby( by = 'iatrogenie').groups[3])
    else:
        class_3_count = 0

    for i in range(len(basic_gr_table)):
        obj_Gr = basic_gr_table.loc[i, 'Granule']

        class_0, class_1, class_2, class_3 = countClasseGr(obj_Gr, info_table)

        basic_gr_table.loc[i, 'coverage_0']  = (class_0 / class_0_count if class_0_count != 0 else 0)
        basic_gr_table.loc[i, 'coverage_1']  = (class_1 / class_1_count if class_1_count != 0 else 0)
        basic_gr_table.loc[i, 'coverage_2']  = (class_2 / class_2_count if class_2_count != 0 else 0)
        basic_gr_table.loc[i, 'coverage_3']  = (class_3 / class_3_count if class_3_count != 0 else 0)


In [55]:
# Définition de la fonction qui calcule l'entropie des granules

def getEntropy(basic_gr_table, info_table):
    
    res = 0

    for i in range(len(basic_gr_table)):

        for j in range(2):
            p_ = basic_gr_table.loc[i, 'confidence_'+str(j)]
            if p_ == 0:
                res += 0
            else:
                res += -( p_ * np.log2(p_) )

        basic_gr_table.loc[i, 'entropy'] = res
        res = 0

Partie III : à partir de ces mesures, on obtiens un premier aperçu de la construction des granules et des différentes mesures associées

In [56]:
# Covering solution ==> Une liste qui contient les "covered solution"  
covering_solution = list()
# Attributes ==> Colonnes des attributs dépendants de nos données en entrée
attributes = []
columns = []

for i in range(t_max):
    attributes.append("treatment"+str(i+1))

    columns.append("treatment"+str(i+1))

# Information table ==> La table qui contient les attributs sur les données et les patients associés
u_info_table = information_table
# Création des granules en fonction des différentes données
u_B_Granules = getFormulasGranules(attributes, information_table )

In [57]:
u_B_Granules

,Formula,Granule
0,treatment1=abiraterone=true,"[6, 9, 21, 24, 29, 42, 50, 59, 62, 64, 69, 79,..."
1,treatment1=acetazolamide=true,"[0, 8, 22, 23, 26, 28, 31, 32, 37, 40, 49, 51,..."
2,treatment1=acide_acetylsaclicylique=true,"[2, 4, 11, 13, 17, 33, 34, 39, 55, 58, 71, 74,..."
3,treatment1=acide_acetylsaclicylique>=3g,"[1, 3, 10, 12, 16, 35, 36, 43, 47, 48, 53, 57,..."
4,treatment1=acide_folinique=true,"[5, 7, 14, 15, 18, 19, 20, 25, 27, 30, 38, 41,..."
5,treatment2=abiraterone=true,"[0, 1, 2, 8, 17, 45, 46, 57, 63, 105, 116, 124..."
6,treatment2=acetazolamide=true,"[10, 12, 27, 34, 43, 53, 66, 67, 69, 72, 78, 8..."
7,treatment2=acide_acetylsaclicylique=true,"[7, 30, 73, 75, 103, 134, 136, 173, 194, 200, ..."
8,treatment2=acide_acetylsaclicylique>=3g,"[55, 76, 96, 109, 111, 143, 152, 168, 171, 176..."
9,treatment2=acide_folinique=true,"[58, 82, 97, 119, 127, 237, 240, 245, 270, 295..."


In [58]:
#On ajoute les différentes mesures à notre Data Frame contenant les granules et leurs formules
#Generality
getGenerality(u_B_Granules, u_info_table) 
#Confidence
getConfidence(u_B_Granules, u_info_table)
#Coverage
getCoverage(u_B_Granules, u_info_table)
#Entropy
getEntropy(u_B_Granules, u_info_table)

In [59]:
u_B_Granules.sort_values(by = ['entropy', 'Generality'], ascending = [True, False], inplace = True)
u_B_Granules.reset_index(drop=True, inplace=True)

u_B_Granules

,Formula,Granule,Generality,confidence_0,confidence_1,confidence_2,confidence_3,coverage_0,coverage_1,coverage_2,coverage_3,entropy
0,treatment2=abiraterone=true,"[0, 1, 2, 8, 17, 45, 46, 57, 63, 105, 116, 124...",0.064,1.000000,0.000000,0.000000,0.000000,0.131687,0.000000,0.000000,0.0,0.000000
1,treatment2=acide_folinique=true,"[58, 82, 97, 119, 127, 237, 240, 245, 270, 295...",0.050,1.000000,0.000000,0.000000,0.000000,0.102881,0.000000,0.000000,0.0,0.000000
2,treatment2=acide_acetylsaclicylique=true,"[7, 30, 73, 75, 103, 134, 136, 173, 194, 200, ...",0.046,1.000000,0.000000,0.000000,0.000000,0.094650,0.000000,0.000000,0.0,0.000000
3,treatment2=acide_acetylsaclicylique>=3g,"[55, 76, 96, 109, 111, 143, 152, 168, 171, 176...",0.052,0.730769,0.000000,0.269231,0.000000,0.078189,0.000000,0.122807,0.0,0.330682
4,treatment1=acide_acetylsaclicylique=true,"[2, 4, 11, 13, 17, 33, 34, 39, 55, 58, 71, 74,...",0.200,0.500000,0.000000,0.000000,0.500000,0.205761,0.000000,0.000000,1.0,0.500000
5,treatment1=acide_acetylsaclicylique>=3g,"[1, 3, 10, 12, 16, 35, 36, 43, 47, 48, 53, 57,...",0.200,0.500000,0.000000,0.500000,0.000000,0.205761,0.000000,0.877193,0.0,0.500000
6,treatment2=acetazolamide=true,"[10, 12, 27, 34, 43, 53, 66, 67, 69, 72, 78, 8...",0.150,0.333333,0.000000,0.666667,0.000000,0.102881,0.000000,0.877193,0.0,0.528321
7,treatment2=anti-inflammatoire_non_stéroïdiens,"[9, 11, 13, 18, 33, 36, 40, 51, 71, 74, 77, 85...",0.164,0.390244,0.000000,0.000000,0.609756,0.131687,0.000000,0.000000,1.0,0.529776
8,treatment2=metoprolol=true,"[3, 6, 25, 32, 39, 50, 59, 64, 65, 79, 93, 107...",0.156,0.358974,0.641026,0.000000,0.000000,0.115226,0.333333,0.000000,0.0,0.941829
9,treatment2=fluorouracil=true,"[4, 14, 15, 16, 19, 20, 24, 29, 31, 35, 37, 38...",0.158,0.367089,0.632911,0.000000,0.000000,0.119342,0.333333,0.000000,0.0,0.948410


In [60]:
#Aperçu du training set initial
information_table

,treatment1,treatment2,iatrogenie
0,acetazolamide=true,abiraterone=true,0
1,acide_acetylsaclicylique>=3g,abiraterone=true,0
2,acide_acetylsaclicylique=true,abiraterone=true,0
3,acide_acetylsaclicylique>=3g,metoprolol=true,0
4,acide_acetylsaclicylique=true,fluorouracil=true,0
...,...,...,...
495,acetazolamide=true,anti-inflammatoire_non_stéroïdiens,0
496,acide_folinique=true,fluorouracil=true,1
497,acide_acetylsaclicylique=true,anti-inflammatoire_non_stéroïdiens,3
498,acetazolamide=true,carbamazepine=true,1


Partie IV : Implémentation de la fonction de construction de l'arbre granulaire

In [61]:
# Construction de l'arbre granulaire
#Cette fonction permet de construire l'arbre granulaire, qui nous sert de modèle pour la détection des règles

level = 1
U_covering_solution = []
#remaining_objs = []
U_remaining_objs =[]

#Initialisation de l'arbre de décision
root = Node("Universe")


def GranuleNetBuilding(inf_table, parent_node, min_entropy_threshold = 0  ):
    global level
    covering_solution = []
    remaining_objs = list(inf_table.index)
    U_remaining_objs = list(inf_table.index)
    
    
    u_B_Granules = getFormulasGranules( attributes, inf_table )
    
    

    #Generality
    getGenerality(u_B_Granules, inf_table)
    #Confidence
    getConfidence(u_B_Granules, inf_table)
    #Coverage
    getCoverage(u_B_Granules, inf_table)
    #Entropy
    getEntropy(u_B_Granules, inf_table)

    u_B_Granules.sort_values(by = ['entropy','Generality'], ascending = [True, False], inplace = True)
    u_B_Granules.reset_index(drop=True, inplace=True)


    min_entropy_indx = u_B_Granules.loc[(u_B_Granules['entropy'] <= min_entropy_threshold)].index
    
    for i in min_entropy_indx:

        
        
        granules1 = u_B_Granules.loc[i,'Granule']
        entropy_val = u_B_Granules.loc[i,'entropy']
        formula = u_B_Granules.loc[i,'Formula']

        not_dup = list(set(granules1)-set(U_covering_solution))

        if not_dup:
            if entropy_val!=0:
                #information_table.iloc[granules, :] #ici information_table est la table global : ce n'est pas la table en argument
                infTable = information_table.iloc[granules1, :]
                level +=1 #on passe à un étage inférieur dans l'Arbre
                GranuleNetBuilding(infTable, parent_node = u_B_Granules.loc[i,'Formula']) #par récursivité, on construit un nouvel arbre à partir de cet étage
                level -=1 #on revient un étage en arrière
            
            else:
                
                #valeur de la classe du premier élément de ce granule[granule d'entropie inférieure au seuil considéré]
                val_out = inf_table.loc[granules1[0],"iatrogenie"] 

                if val_out != 0:
                    b_node = Node(formula, class_ = val_out, parent = parent_node, )
                    print("\t"*level,"LEVEL =====> ", level)
                    print("\t"*level,"Formula: ", u_B_Granules.loc[i,'Formula'])
                    print("\t"*level,"Granules: ", u_B_Granules.loc[i,'Granule'])
                
                #On ajoute ces éléments d'entropie minimale aux solutions
                covering_solution.extend(list(u_B_Granules.loc[i,'Granule'])) 
                #On ajoute ces éléments d'entropie minimale aux solutions globales
                U_covering_solution.extend(list(u_B_Granules.loc[i,'Granule'])) 
                #on retire les index des sol d'entropie nulles aux index des données qu'on analyse
                remaining_objs = set(inf_table.index) - set(covering_solution) 
                
                U_remaining_objs = set(inf_table.index) - set(covering_solution) 
        
        else:
            covering_solution.extend(list(u_B_Granules.loc[i,'Granule']))
            U_covering_solution.extend(list(u_B_Granules.loc[i,'Granule']))
            remaining_objs = set(inf_table.index) - set(covering_solution)

            U_remaining_objs = set(inf_table.index) - set(covering_solution)


        u_B_Granules.drop(i, inplace = True)

    
    
    while len(remaining_objs) > 0:
        
        ##Calcul des indices de Jaccar
        min_coverage_index = u_B_Granules.index[0]
        coverage_inter = len(set(u_B_Granules.loc[min_coverage_index, 'Granule']) & set(remaining_objs))#On cherche si ce granule est d'entropie 0 ou pas
        coverage_union = len(set(u_B_Granules.loc[min_coverage_index, 'Granule']) | set(remaining_objs))
        coverage_val = coverage_inter / coverage_union

        for i in u_B_Granules.index:
            coverage_inter = len(set(u_B_Granules.loc[i, 'Granule']) & set(remaining_objs))
            coverage_union = len(set(u_B_Granules.loc[i, 'Granule']) | set(remaining_objs))
            
            cov_temp = coverage_inter / coverage_union
            
            if cov_temp > coverage_val: ## On cherche l'index maximisant ce ratio
                    min_coverage_index = i
                    coverage_val = cov_temp 

        granules = u_B_Granules.loc[min_coverage_index, 'Granule']
        not_dup = list(set(granules)-set(covering_solution))
        formula = u_B_Granules.loc[min_coverage_index,'Formula']

        print("\t"*level,"LEVEL =====> ", level)
        print("\t"*level,"Formula: ", u_B_Granules.loc[min_coverage_index,'Formula'])
        print("\t"*level,"Granules: ", u_B_Granules.loc[min_coverage_index,'Granule'])

        # On ajoute ce granule à la liste des covering solution
        covering_solution.extend(list(u_B_Granules.loc[min_coverage_index,'Granule'])) 
        remaining_objs = set(inf_table.index) - set(covering_solution)

        U_remaining_objs = set(inf_table.index) - set(covering_solution)

            
        u_B_Granules.drop(min_coverage_index, inplace=True)

        infTable = information_table.iloc[granules, :]

        level += 1
        rt = Node(formula, parent = parent_node)

        GranuleNetBuilding(infTable, parent_node=rt)
        level -= 1

        
    ##############################################

In [62]:
#on construit ici l'arbre granulaire
GranuleNetBuilding(information_table,root)

	 LEVEL =====>  1
	 Formula:  treatment1=abiraterone=true
	 Granules:  [6, 9, 21, 24, 29, 42, 50, 59, 62, 64, 69, 79, 84, 93, 96, 106, 107, 108, 109, 113, 115, 121, 128, 134, 136, 163, 165, 169, 170, 174, 176, 181, 183, 187, 195, 197, 202, 208, 209, 210, 211, 215, 219, 228, 246, 258, 264, 271, 272, 273, 283, 286, 292, 295, 298, 299, 300, 301, 302, 304, 305, 312, 314, 319, 323, 326, 329, 340, 343, 353, 358, 364, 368, 370, 371, 372, 378, 388, 393, 395, 399, 411, 414, 415, 419, 437, 451, 458, 459, 460, 464, 465, 472, 477, 482, 484, 487, 488, 490, 492]
		 LEVEL =====>  2
		 Formula:  treatment2=metoprolol=true
		 Granules:  [6, 50, 59, 64, 79, 93, 107, 108, 121, 163, 174, 181, 187, 195, 208, 210, 219, 246, 264, 271, 272, 273, 286, 292, 300, 304, 312, 314, 319, 323, 329, 340, 343, 353, 364, 368, 371, 372, 411, 414, 415, 451, 458, 460, 464, 465, 472, 477, 490, 492]
	 LEVEL =====>  1
	 Formula:  treatment1=acide_acetylsaclicylique=true
	 Granules:  [2, 4, 11, 13, 17, 33, 34, 39, 55, 58, 71, 7

On récupère maintenant l'arbre granulaire par sa raçine : root [Element de la bibliothèqe Big tree]

In [63]:
from bigtree import tree_to_dot
from bigtree import tree_to_dataframe

#Toute l'information de l'arbre granulaire est contenue dans root
root.show(attr_list = ["class_"])

#Fichier PNG contenant l'arbre
graph = tree_to_dot(root,node_attr="node_attr")
graph.write_png("tree_iatro.png")



Universe
├── treatment1=abiraterone=true
│   └── treatment2=metoprolol=true [class_=1]
├── treatment1=acide_acetylsaclicylique=true
│   └── treatment2=anti-inflammatoire_non_stéroïdiens [class_=3]
├── treatment1=acetazolamide=true
│   ├── treatment2=carbamazepine=true [class_=1]
│   └── treatment2=acide_acetylsaclicylique>=3g [class_=2]
├── treatment1=acide_acetylsaclicylique>=3g
│   └── treatment2=acetazolamide=true [class_=2]
└── treatment1=acide_folinique=true
    └── treatment2=fluorouracil=true [class_=1]


Partie 5 : Entrée de nouvelles données et détermination des règle en parcourant l'arbre granulaire

On donne de nouvelles données à l'IA qui va déterminer si elles forment une règle iatrogénique

In [64]:
# nouvelle donnée en entrée [traitements]
data = [['fluorouracil=true','acide_folinique=true']]

In [65]:
#changement de format
df = pd.DataFrame(data=data, columns=columns)

#mise en place des attributs du patient
attribut = []
for i in range(t_max):
    attribut.append('treatment'+str(i+1))


df

,treatment1,treatment2
0,fluorouracil=true,acide_folinique=true


In [66]:
#Fonction qui crée un règle 

def creationNewRule(fact_list, conclusion, rule_name):
    newRule = {rule_name :{'condition' :fact_list, 'consequence': conclusion } }
    #test = newRule in rules_base.rules #Peut être pas assez rigoureux
    #rules.update(newRule)

    return True

#Fonction qui nous permet de récupérer les formules (format) associées aux données en entrée
def getTestFormulas(test_table, attribut):
    all_formula_test = []
    for i in range(len(test_table)):
        p_i_attribute = []
        for attr in attribut:
            p_i_attribute.append( attr + "=" + test_table.loc[i, attr])
        all_formula_test.append(p_i_attribute)
    return all_formula_test

In [67]:
#on récupère les attributs associés aux données
data_attributes = getTestFormulas(df, attribut)[0] #On récupère les attributs de la première personne
print(data_attributes)

['treatment1=fluorouracil=true', 'treatment2=acide_folinique=true']


Nous implémentons une fonction qui nous permet de reparcourir l'arbre granulaire : permet de déterminer si les données en entrées forment une règle

In [68]:
def FindFormulasInTree(formula_test, root,fact_list):
    
    X = False  # L'état de la recherche
    copy = formula_test.copy()
    detected_class = -1 #classe abstraite --> aucune classe iatrogénique détectée
    while not X:
        parent = root
        children = list(parent.children)
        for form in copy:
            
            for child in children:
                
                if retirerChiffre(form.strip()) == retirerChiffre(child.node_name.strip()): #permet d'éviter la prise en compte de l'ordre dans l'entrée des traitement
                    if list(child.children):
                        copy.remove(form)

                        fact_list.append(child.node_name.strip()) 
                        detected_class = FindFormulasInTree(copy, parent,fact_list)
                        if detected_class != -1:  # Si detected_class est différent de -1, on sort immédiatement
                            return detected_class
                        
                        
                        parent = child
                        detected_class = FindFormulasInTree(copy, parent,fact_list)  # Appel récursif pour mettre à jour detected_class
                        if detected_class != -1:  # Si detected_class est différent de -1, on sort immédiatement
                            return detected_class
                    else:
                        
                        fact_list.append(child.node_name.strip())
                        X = True
                        detected_class = child.class_
                        
        if not X:
            return detected_class
    
    return detected_class


In [69]:
##on crée une nouvelle liste de fait vide : va nous servir à créer une nouvelle règle potentiellement
fact_list=[]

##Détection de la règle
result = FindFormulasInTree(data_attributes,root,fact_list) ##On cherche 

print("la classe iatrogénique est :", result)

la classe iatrogénique est : 1


ça y est ! l'IA a bien retrouvé une des règles à partir de laquelle nous avions crée notre DataSet Initiam

cela prouve qu'elle serait capable de trouver de nouvelles règles non connues dans un data set bien nettoyé